# see if strep and staph actually anti-correlate

In [259]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import biom
from biom import load_table
import numpy as np
from scipy.stats import spearmanr


In [260]:
# Load the metadata
metadata_path = '../Data/Metadata/updated_clean_ant_skin_metadata.tab'
metadata = pd.read_csv(metadata_path, sep='\t')

metadata['#sample-id'] = metadata['#sample-id'].str.replace('_', '')
# Set Sample-ID as the index for the metadata dataframe 
metadata = metadata.set_index('#sample-id')


# Create group column based on case_type to simplify group names
metadata['group'] = metadata['case_type'].map({
    'case-lesional skin': 'skin-ADL',
    'case-nonlesional skin': 'skin-ADNL', 
    'control-nonlesional skin': 'skin-H',
    'case-anterior nares': 'nares-AD',
    'control-anterior nares': 'nares-H'
})

metadata

,PlateNumber,PlateLocation,i5,i5Sequence,i7,i7Sequence,identifier,Sequence,Plate ID,Well location,...,sex,enrolment_date,enrolment_season,hiv_exposure,hiv_status,household_size,o_scorad,FWD_filepath,REV_filepath,group
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STL,1,A1,SA501,ATCGTACG,SA701,CGAGAGTT,SA701SA501,CGAGAGTT-ATCGTACG,1.010000e+21,A1,...,male,4/16/2015,Autumn,Unexposed,negative,4.0,40,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL
900221,1,B1,SA502,ACTATCTG,SA701,CGAGAGTT,SA701SA502,CGAGAGTT-ACTATCTG,1.010000e+21,B1,...,female,8/11/2015,Winter,Unexposed,negative,7.0,34,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL
Ca010EBL,1,C1,SA503,TAGCGAGT,SA701,CGAGAGTT,SA701SA503,CGAGAGTT-TAGCGAGT,1.010000e+21,C1,...,female,11/20/2014,Spring,Unexposed,negative,7.0,21,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL
900460,1,D1,SA504,CTGCGTGT,SA701,CGAGAGTT,SA701SA504,CGAGAGTT-CTGCGTGT,1.010000e+21,D1,...,female,9/23/2015,Spring,Unexposed,NaN,4.0,40,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL
900051,1,E1,SA505,TCATCGAG,SA701,CGAGAGTT,SA701SA505,CGAGAGTT-TCATCGAG,1.010000e+21,E1,...,male,4/21/2015,Autumn,Unexposed,negative,7.0,41,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ca006ONL2,6,H1,SA508,GACACCGT,SB701,CTCGACTT,SB701SA508,CTCGACTT-GACACCGT,1.010000e+21,H1,...,female,3/25/2015,Autumn,Unexposed,negative,3.0,34,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL
Ca006ONNL,6,F2,SA506,CGTGAGTG,SB702,CGAAGTAT,SB702SA506,CGAAGTAT-CGTGAGTG,1.010000e+21,F2,...,female,3/25/2015,Autumn,Unexposed,negative,3.0,34,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADNL
Ca006ONNL2,6,H2,SA508,GACACCGT,SB702,CGAAGTAT,SB702SA508,CGAAGTAT-GACACCGT,1.010000e+21,H2,...,female,3/25/2015,Autumn,Unexposed,negative,3.0,34,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADNL


In [261]:
metadata.columns

Index(['PlateNumber', 'PlateLocation', 'i5', 'i5Sequence', 'i7', 'i7Sequence',
       'identifier', 'Sequence', 'Plate ID', 'Well location', 'Volume (ul)',
       'Lysozyme pretreatment', 'DNA extraction method', 'Purification method',
       'Date of DNA extraction', 'pid', 'case_type', 'participant', 'area',
       'sample_type', 'specimen', 'age_months', 'sex', 'enrolment_date',
       'enrolment_season', 'hiv_exposure', 'hiv_status', 'household_size',
       'o_scorad', 'FWD_filepath', 'REV_filepath', 'group'],
      dtype='object')

In [262]:
metadata['case_type'].value_counts()

case_type
case-nonlesional skin       111
case-anterior nares         108
case-lesional skin          107
control-anterior nares       89
control-nonlesional skin     87
Name: count, dtype: int64

In [263]:
# Create skin_ad_status column based on case_type
metadata['skin_ad_status'] = metadata['case_type'].apply(lambda x: 
    'AD' if x.endswith('skin') and x.startswith('case') 
    else 'Healthy' if x.endswith('skin') and x.startswith('control')
    else None)

metadata

,PlateNumber,PlateLocation,i5,i5Sequence,i7,i7Sequence,identifier,Sequence,Plate ID,Well location,...,enrolment_date,enrolment_season,hiv_exposure,hiv_status,household_size,o_scorad,FWD_filepath,REV_filepath,group,skin_ad_status
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STL,1,A1,SA501,ATCGTACG,SA701,CGAGAGTT,SA701SA501,CGAGAGTT-ATCGTACG,1.010000e+21,A1,...,4/16/2015,Autumn,Unexposed,negative,4.0,40,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL,AD
900221,1,B1,SA502,ACTATCTG,SA701,CGAGAGTT,SA701SA502,CGAGAGTT-ACTATCTG,1.010000e+21,B1,...,8/11/2015,Winter,Unexposed,negative,7.0,34,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL,AD
Ca010EBL,1,C1,SA503,TAGCGAGT,SA701,CGAGAGTT,SA701SA503,CGAGAGTT-TAGCGAGT,1.010000e+21,C1,...,11/20/2014,Spring,Unexposed,negative,7.0,21,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL,AD
900460,1,D1,SA504,CTGCGTGT,SA701,CGAGAGTT,SA701SA504,CGAGAGTT-CTGCGTGT,1.010000e+21,D1,...,9/23/2015,Spring,Unexposed,NaN,4.0,40,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL,AD
900051,1,E1,SA505,TCATCGAG,SA701,CGAGAGTT,SA701SA505,CGAGAGTT-TCATCGAG,1.010000e+21,E1,...,4/21/2015,Autumn,Unexposed,negative,7.0,41,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL,AD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ca006ONL2,6,H1,SA508,GACACCGT,SB701,CTCGACTT,SB701SA508,CTCGACTT-GACACCGT,1.010000e+21,H1,...,3/25/2015,Autumn,Unexposed,negative,3.0,34,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL,AD
Ca006ONNL,6,F2,SA506,CGTGAGTG,SB702,CGAAGTAT,SB702SA506,CGAAGTAT-CGTGAGTG,1.010000e+21,F2,...,3/25/2015,Autumn,Unexposed,negative,3.0,34,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADNL,AD
Ca006ONNL2,6,H2,SA508,GACACCGT,SB702,CGAAGTAT,SB702SA508,CGAAGTAT-GACACCGT,1.010000e+21,H2,...,3/25/2015,Autumn,Unexposed,negative,3.0,34,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADNL,AD


In [264]:
# Read in table at ASV level
biom_path = '../Data/Tables/Absolute_Abundance_Tables/df_16S_filtered_feature_table_rare_Genus_absolute.biom'
biom_tbl = load_table(biom_path)
df = pd.DataFrame(biom_tbl.to_dataframe().T)

# delete the prefix from the index
df.index = df.index.str.replace('15564.', '')

In [265]:
# def log_transform_abundance(df, pseudocount=1):
#     """
#     Apply log10(x + pseudocount) to all numeric columns in the DataFrame.
#     """
#     df_log = df.copy()
#     for col in df_log.select_dtypes(include=[np.number]).columns:
#         df_log[col] = np.log10(df_log[col] + pseudocount)
#     return df_log

# df = log_transform_abundance(df)
# df

In [266]:
# Convert to dense format
def convert_sparse_to_dense(df):
    df_dense = df.copy()
    for col in df_dense.columns:
        if pd.api.types.is_sparse(df_dense[col].dtype):
            df_dense[col] = df_dense[col].sparse.to_dense()
    return df_dense

# Convert your df to dense
df = convert_sparse_to_dense(df)
df

/var/folders/22/yck9vwx53w1c38tvj_c0_tz00000gn/T/ipykernel_61292/3258987142.py:5: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if pd.api.types.is_sparse(df_dense[col].dtype):


,g__Streptococcus,g__Staphylococcus,g__Haemophilus_D_734546,g__Corynebacterium,g__,g__Acinetobacter,g__Prevotella,g__Micrococcus,g__Dolosigranulum,g__Moraxella_A,...,g__Aerococcus,g__Tepidiphilus,g__Pseudomonas_K,g__CADDYX01,g__Pseudomonas_S,g__Tetragenococcus,g__Thermicanus,g__Thermoanaerobacterium,g__Thermus_A,g__Nakamurella
900344,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
900459,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
900221,2.0,175.0,0.0,7.0,7.0,82.0,3.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
900570,361.0,710.0,2.0,37.0,510.0,55.0,645.0,0.0,0.0,43.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
900092,2920.0,0.0,0.0,46.0,20.0,0.0,0.0,2.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003972,674.0,380.0,47.0,45.0,113.0,24.0,148.0,29.0,0.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
900097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
900498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
900276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [267]:
# Calculate the percentage of non-zero values for each feature
presence_threshold = 0.10  # 10%
feature_presence = (df != 0).mean()

# Keep only features that appear in more than 10% of samples
df = df.loc[:, feature_presence >= presence_threshold]
df

,g__Streptococcus,g__Staphylococcus,g__Haemophilus_D_734546,g__Corynebacterium,g__,g__Acinetobacter,g__Prevotella,g__Micrococcus,g__Dolosigranulum,g__Moraxella_A,...,g__Rothia,g__Capnocytophaga,g__Anaerococcus,g__Finegoldia,g__Sphingomonas_L_486704,g__Brevundimonas,g__Alloprevotella,g__Nocardioides_A_392796,g__Telluria_573210,g__Pauljensenia
900344,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
900459,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
900221,2.0,175.0,0.0,7.0,7.0,82.0,3.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
900570,361.0,710.0,2.0,37.0,510.0,55.0,645.0,0.0,0.0,43.0,...,0.0,0.0,4.0,0.0,0.0,3.0,4.0,12.0,0.0,4.0
900092,2920.0,0.0,0.0,46.0,20.0,0.0,0.0,2.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003972,674.0,380.0,47.0,45.0,113.0,24.0,148.0,29.0,0.0,12.0,...,13.0,12.0,11.0,1.0,1.0,0.0,48.0,0.0,7.0,4.0
900097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
900498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
900276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [268]:
def convert_to_relative_abundance(df):
    """
    Convert a sample-by-feature DataFrame to relative abundance per sample,
    safely handling rows with zero total counts (which produce NaNs).
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame where rows are samples and columns are features
    
    Returns:
    --------
    pandas.DataFrame
        Relative abundance DataFrame (rows with zero counts remain 0.0)
    """
    row_sums = df.sum(axis=1)
    # Avoid division by zero
    rel_abundance = df.div(row_sums.replace(0, np.nan), axis=0)
    # Replace any resulting NaNs with 0
    rel_abundance = rel_abundance.fillna(0)
    return rel_abundance

df = convert_to_relative_abundance(df)

In [269]:
# Map the 'group' column from metadata to df based on matching index
df['skin_ad_status'] = metadata.loc[df.index, 'skin_ad_status']
df = df[(df['skin_ad_status'] == 'AD') | (df['skin_ad_status'] == 'Healthy')]

# df = df[(df['group'] == 'skin-ADL') | (df['group'] == 'skin-ADNL')]
# df = df[(df['group'] == 'skin-ADL')]
# df = df[(df['group'] == 'skin-ADNL')]

df

,g__Streptococcus,g__Staphylococcus,g__Haemophilus_D_734546,g__Corynebacterium,g__,g__Acinetobacter,g__Prevotella,g__Micrococcus,g__Dolosigranulum,g__Moraxella_A,...,g__Capnocytophaga,g__Anaerococcus,g__Finegoldia,g__Sphingomonas_L_486704,g__Brevundimonas,g__Alloprevotella,g__Nocardioides_A_392796,g__Telluria_573210,g__Pauljensenia,skin_ad_status
900344,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,Healthy
900221,0.000668,0.058411,0.000000,0.002336,0.002336,0.027370,0.001001,0.001335,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000668,0.000000,0.00000,0.000000,AD
900570,0.127292,0.250353,0.000705,0.013047,0.179831,0.019394,0.227433,0.000000,0.0,0.015162,...,0.000000,0.001410,0.000000,0.000000,0.001058,0.001410,0.004231,0.00000,0.001410,AD
900129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,AD
900321,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,Healthy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003972,0.239772,0.135183,0.016720,0.016009,0.040199,0.008538,0.052650,0.010317,0.0,0.004269,...,0.004269,0.003913,0.000356,0.000356,0.000000,0.017076,0.000000,0.00249,0.001423,AD
900097,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,AD
900498,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,AD
900276,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,AD


In [270]:
# Set styling for better readability
# sns.set_context("talk")
# sns.set_style("whitegrid")

# Jointplot: Streptococcus vs Staphylococcus, colored by zone
g = sns.jointplot(
    data=df,
    x=' g__Streptococcus',
    y=' g__Staphylococcus',
    hue='skin_ad_status',
    kind='scatter',
    alpha=0.7,
    height=8,
    marginal_kws=dict(common_norm=False, fill=True)
)

# Set limits
g.ax_joint.set_xlim(-0.05, 0.8)
g.ax_joint.set_ylim(-0.05, 1.05)

# Customize tick labels to hide -0.05 and 1.05
xticks = [tick for tick in g.ax_joint.get_xticks() if tick > -0.05]
yticks = [tick for tick in g.ax_joint.get_yticks() if tick > -0.05]

# xticks = [tick for tick in g.ax_joint.get_xticks() if tick < 1.05]
yticks = [tick for tick in g.ax_joint.get_yticks() if tick < 1.05]

g.ax_joint.set_xticks(xticks)
g.ax_joint.set_yticks(yticks)

# Add title and save
g.fig.suptitle("Streptococcus vs Staphylococcus Abundance by Skin Zone", fontsize=16, y=1.05)
plt.tight_layout()
plt.savefig("../Plots/Analysis_figures/Streptococcus_vs_Staphylococcus_abundance.png", dpi=600)

/Users/yangchen/miniforge3/envs/qiime2-metagenome-2024.10/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/Users/yangchen/miniforge3/envs/qiime2-metagenome-2024.10/lib/python3.10/site-packages/seaborn/_oldcore.py:1075: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  data_subset = grouped_data.get_group(pd_key)
/Users/yangchen/miniforge3/envs/qiime2-metagenome-2024.10/lib/python3.10/site-packages/seaborn/_oldcore.py:1075: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  data_subset = grouped

In [271]:
df

,g__Streptococcus,g__Staphylococcus,g__Haemophilus_D_734546,g__Corynebacterium,g__,g__Acinetobacter,g__Prevotella,g__Micrococcus,g__Dolosigranulum,g__Moraxella_A,...,g__Capnocytophaga,g__Anaerococcus,g__Finegoldia,g__Sphingomonas_L_486704,g__Brevundimonas,g__Alloprevotella,g__Nocardioides_A_392796,g__Telluria_573210,g__Pauljensenia,skin_ad_status
900344,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,Healthy
900221,0.000668,0.058411,0.000000,0.002336,0.002336,0.027370,0.001001,0.001335,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000668,0.000000,0.00000,0.000000,AD
900570,0.127292,0.250353,0.000705,0.013047,0.179831,0.019394,0.227433,0.000000,0.0,0.015162,...,0.000000,0.001410,0.000000,0.000000,0.001058,0.001410,0.004231,0.00000,0.001410,AD
900129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,AD
900321,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,Healthy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003972,0.239772,0.135183,0.016720,0.016009,0.040199,0.008538,0.052650,0.010317,0.0,0.004269,...,0.004269,0.003913,0.000356,0.000356,0.000000,0.017076,0.000000,0.00249,0.001423,AD
900097,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,AD
900498,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,AD
900276,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,AD


In [272]:
# Filter out zero values
df_corr = df[(df[' g__Streptococcus'] > 0) & (df[' g__Staphylococcus'] > 0)]

# Compute Spearman correlation
rho, p = spearmanr(df_corr[' g__Streptococcus'], df_corr[' g__Staphylococcus'])

# Plot with regression line
plt.figure(figsize=(7, 7))
sns.regplot(
    x=' g__Streptococcus',
    y=' g__Staphylococcus',
    data=df_corr,
    scatter_kws={'alpha': 0.7},
    line_kws={'color': 'red'},
    ci=None
)

plt.xlabel("Streptococcus (relative abundance)")
plt.ylabel("Staphylococcus (relative abundance)")
plt.title(f"Streptococcus vs Staphylococcus Abundance\nSpearman r = {rho:.2f}, p = {p:.5f}")
plt.tight_layout()
plt.savefig("../Plots/Analysis_figures/strep_vs_staph_abundance_with_fit.png", dpi=600)